In [ ]:
import pandas as pd

import importlib

import file_loader
importlib.reload(file_loader)

from file_loader import FileLoader

# Percorso del file CSV (usa path relativo)
csv_path = "../data/input/determinazioni_amianto_2024.csv"
# Leggi il CSV con pandas
try:
    # encoding='latin1'  → Vecchi file Windows/Europa occidentale
    # encoding='cp1252'  → Variante Windows, gestisce più simboli (€ ecc.)
    # encoding='utf-8-sig' → Da usare se la prima colonna ha caratteri strani (BOM UTF-8), tipico da Excel
    # Provare diversi encoding se si vedono caratteri strani nei dati/colonne!
    loader = FileLoader(csv_path,';','utf-8-sig')
    df = loader.load()
    print("Dati caricati correttamente:")
    print(df.head())
except Exception as e:
    print(f"Errore nel caricamento: {e}")
    raise



Dati caricati correttamente:
   codice_record  id_punto_campionamento  \
0            265                     200   
1            282                     201   
2            283                     202   
3            408                     203   
4            409                     204   

                                          nome_punto               Comune  \
0  punto1 - cono di deiezione nei pressi del matt...  SAN SEVERINO LUCANO   
1  punto 2 - piazzola di sosta sottostante un pic...  SAN SEVERINO LUCANO   
2            punto 3 - strada sterrata per 3 confini  SAN SEVERINO LUCANO   
3            loc. Rondinelle - complanare nord MT-BA               MATERA   
4            loc. Rondinelle - complanare nord MT-BA               MATERA   

   id_campionamento                               descrizione_campione  \
0               580         cono di deiezione nei pressi del mattatoio   
1               581  piazzola di sosta sottostante un piccolo front...   
2               582  